# Topog form stress using MOM025 DATA

Recent update 11 Sept 2017

In [8]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import os
import pickle
from numpy import math

import pandas as pd
import xarray as xr
from glob import glob
from mpl_toolkits.basemap import Basemap, shiftgrid


# Load Data

In [24]:
## Load data

%time
OceanFile1 = '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55_ryf_spinup7/output100/ocean/ocean_grid.nc'
dsx1 = xr.open_dataset(OceanFile1, engine='netcdf4')
kmt = dsx1.kmt.isel(yt_ocean=217).astype(np.int64)  # "number of depth levels on t-grid"
depth = -dsx1.ht.isel(yt_ocean=217)   ## H
dxu = dsx1.dxu ## dxu

OceanFile2 =  '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55_ryf_spinup7/output100/ocean/ocean_month.nc'
dsx2 = xr.open_dataset(OceanFile2, engine='netcdf4')
taux = dxu*dsx2.tau_x.mean('time')  ## tau_x
pbot = dsx2.pbot_t.mean('time').isel(yt_ocean=217)     ## p_bot
eta0 = dsx2.eta_t.mean('time') ## \eta
eta = dsx2.sea_level.mean('time') ## sea_level

OceanFile3 =  '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55_ryf_spinup7/output100/ocean/ocean.nc'
dsx3 = xr.open_dataset(OceanFile3, engine='netcdf4')
rho = dsx3.rho.mean('time').isel(yt_ocean=217)        ## \rho in-situ
dz = dsx3.dzt.mean('time').isel(yt_ocean=217)      ## t-cell thickness
g = 9.8

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs


## Step 1: 
## define water thickness as h: $h = -\int dz$
## define bottom pressure as p: $p = \int \rho*g*dz$

In [25]:
h = -dz.sum('st_ocean')

dp = rho*g*dz
p = dp.sum('st_ocean')

## Step 2:
## Calculate $\sum p*\Delta z$

In [26]:
a=0
z=np.zeros(kmt.shape).astype(np.int64) 
z[:]=kmt[:].values-1
dp1 = 0

pb=np.zeros(p.shape)
pb[:]=p[:].values

for x in range(1392,1411):
    dkmt1=kmt.isel(xt_ocean=x)-kmt.isel(xt_ocean=x-1)
    if dkmt1<0:
        dz1=dz.isel(xt_ocean=x-1).isel(st_ocean=z[x])-dz.isel(xt_ocean=x).isel(st_ocean=z[x])
        rho1=rho.isel(xt_ocean=x-1).isel(st_ocean=z[x])
        dp1= dp1-pb[x]*dz1 -rho1*g*dz1**2/2
    elif dkmt1>0:
        dz1=dz.isel(xt_ocean=x).isel(st_ocean=z[x])
        rho1=rho.isel(xt_ocean=x).isel(st_ocean=z[x])
        dp1= dp1+pb[x]*dz1 -rho1*g*dz1**2/2        

In [27]:
dp2 = 0

for y in range(1391,1410):
    dkmt2=kmt.isel(xt_ocean=y+1)-kmt.isel(xt_ocean=y) 
    if dkmt2==0:
        dz2=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y])-dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y])
        dp2= dp2+pb[y]*dz2 +rho2*g*dz2**2/2
    elif dkmt2==1:
        dz2=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y])-dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y])
        dp2= dp2+pb[y]*dz2 +rho2*g*dz2**2/2
    elif dkmt2==-1:
        dz2=dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y).isel(st_ocean=z[y])
        dp2= dp2-pb[y]*dz2 +rho2*g*dz2**2/2
    elif dkmt2==2:
        dz2=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y])-dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y])
        dz3=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+1)        
        rho3=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+1)
        dp2= dp2+pb[y]*(dz2+dz3)+rho2*g*dz2*dz3 
        +rho2*g*dz2**2/2+ rho3*g*dz3**2/2  
    elif dkmt2==-2:
        dz2=dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y).isel(st_ocean=z[y])
        dz3=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-1)        
        rho3=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-1)
        dp2= dp2-pb[y]*(dz2+dz3)+rho2*g*dz2*dz3 
        +rho2*g*dz2**2/2+ rho3*g*dz3**2/2          
    elif dkmt2==3:
        dz2=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y])-dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y])
        dz3=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+1)        
        rho3=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+1)
        dz4=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+2)        
        rho4=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+2)        
        dp2= dp2+pb[y]*(dz2+dz3+dz4)+rho2*g*dz2*(dz3+dz4)
        +rho3*g*dz3*dz4 +rho2*g*dz2**2/2+ rho3*g*dz3**2/2
        + rho4*g*dz4**2/2
    elif dkmt2==-3:
        dz2=dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y).isel(st_ocean=z[y])
        dz3=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-1)        
        rho3=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-1)
        dz4=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-2)        
        rho4=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-2)        
        dp2= dp2-pb[y]*(dz2+dz3+dz4)+rho2*g*dz2*(dz3+dz4)
        +rho3*g*dz3*dz4 +rho2*g*dz2**2/2+ rho3*g*dz3**2/2
        + rho4*g*dz4**2/2        
    elif dkmt2==4:
        dz2=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y])-dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y])
        dz3=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+1)        
        rho3=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+1)
        dz4=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+2)        
        rho4=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+2)      
        dz5=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+3)        
        rho5=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+3)        
        dp2= dp2+pb[y]*(dz2+dz3+dz4+dz5)
        +rho2*g*dz2*(dz3+dz4+dz5)+rho3*g*dz3*(dz4+dz5)
        +rho4*g*dz4*(dz5)+rho2*g*dz2**2/2
        +rho3*g*dz3**2/2+rho4*g*dz4**2/2
        + rho5*g*dz5**2/2  
    elif dkmt2==-4:
        dz2=dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y).isel(st_ocean=z[y])
        dz3=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-1)        
        rho3=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-1)
        dz4=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-2)        
        rho4=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-2)      
        dz5=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-3)        
        rho5=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-3)        
        dp2= dp2-pb[y]*(dz2+dz3+dz4+dz5)+rho2*g*dz2*(dz3+dz4+dz5)+rho3*g*dz3*(dz4+dz5)+rho4*g*dz4*(dz5)+rho2*g*dz2**2/2+rho3*g*dz3**2/2+rho4*g*dz4**2/2+ rho5*g*dz5**2/2                
    elif dkmt2==5:
        dz2=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y])-dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y])
        dz3=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+1)        
        rho3=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+1)
        dz4=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+2)        
        rho4=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+2)      
        dz5=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+3)        
        rho5=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+3) 
        dz6=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+4)        
        rho6=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+4)        
        dp2= dp2+pb[y]*(dz2+dz3+dz4+dz5+dz6)+rho2*g*dz2*(dz3+dz4+dz5+dz6)+rho3*g*dz3*(dz4+dz5+dz6)+rho4*g*dz4*(dz5+dz6)+rho5*g*dz5*(dz6)+rho2*g*dz2**2/2+rho3*g*dz3**2/2+rho4*g*dz4**2/2+ rho5*g*dz5**2/2+ rho6*g*dz6**2/2   
    elif dkmt2==-5:
        dz2=dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y).isel(st_ocean=z[y])
        dz3=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-1)        
        rho3=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-1)
        dz4=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-2)        
        rho4=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-2)      
        dz5=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-3)        
        rho5=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-3) 
        dz6=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-4)        
        rho6=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-4)        
        dp2= dp2-pb[y]*(dz2+dz3+dz4+dz5+dz6)+rho2*g*dz2*(dz3+dz4+dz5+dz6)+rho3*g*dz3*(dz4+dz5+dz6)+rho4*g*dz4*(dz5+dz6)+rho5*g*dz5*(dz6)+rho2*g*dz2**2/2+rho3*g*dz3**2/2+rho4*g*dz4**2/2+ rho5*g*dz5**2/2+ rho6*g*dz6**2/2                 
    elif dkmt2==9:
        dz2=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y])-dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y])
        dz3=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+1)        
        rho3=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+1)
        dz4=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+2)        
        rho4=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+2)      
        dz5=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+3)        
        rho5=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+3) 
        dz6=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+4)        
        rho6=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+4) 
        dz7=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+5)        
        rho7=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+5) 
        dz8=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+6)        
        rho8=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+6) 
        dz9=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+7)        
        rho9=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+7) 
        dz10=dz.isel(xt_ocean=y+1).isel(st_ocean=z[y]+8)        
        rho10=rho.isel(xt_ocean=y+1).isel(st_ocean=z[y]+8)         
        dp2= dp2+pb[y]*(dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10)+rho2*g*dz2*(dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10)+rho3*g*dz3*(dz4+dz5+dz6+dz7+dz8+dz9+dz10)+rho4*g*dz4*(dz5+dz6+dz7+dz8+dz9+dz10)+rho5*g*dz5*(dz6+dz7+dz8+dz9+dz10)+rho6*g*dz6*(dz7+dz8+dz9+dz10)+rho7*g*dz7*(dz8+dz9+dz10)+rho8*g*dz8*(dz9+dz10)+rho9*g*dz9*(dz10)+rho2*g*dz2**2/2+rho3*g*dz3**2/2+rho4*g*dz4**2/2+ rho5*g*dz5**2/2+ rho6*g*dz6**2/2+rho7*g*dz7**2/2+ rho8*g*dz8**2/2+ rho9*g*dz9**2/2+ rho10*g*dz10**2/2    
    elif dkmt2==-7:
        dz2=dz.isel(xt_ocean=y).isel(st_ocean=z[y])
        rho2=rho.isel(xt_ocean=y).isel(st_ocean=z[y])
        dz3=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-1)        
        rho3=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-1)
        dz4=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-2)        
        rho4=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-2)      
        dz5=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-3)        
        rho5=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-3) 
        dz6=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-4)        
        rho6=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-4) 
        dz7=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-5)        
        rho7=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-5) 
        dz8=dz.isel(xt_ocean=y).isel(st_ocean=z[y]-6)        
        rho8=rho.isel(xt_ocean=y).isel(st_ocean=z[y]-6)         
        dp2= dp2-pb[y]*(dz2+dz3+dz4+dz5+dz6+dz7+dz8)+rho2*g*dz2*(dz3+dz4+dz5+dz6+dz7+dz8)+rho3*g*dz3*(dz4+dz5+dz6+dz7+dz8)+rho4*g*dz4*(dz5+dz6+dz7+dz8)+rho5*g*dz5*(dz6+dz7+dz8)+rho6*g*dz6*(dz7+dz8)+rho7*g*dz7*(dz8)+rho2*g*dz2**2/2+rho3*g*dz3**2/2+rho4*g*dz4**2/2+ rho5*g*dz5**2/2+ rho6*g*dz6**2/2+rho7*g*dz7**2/2+ rho8*g*dz8**2/2         


## Step 3: output the result

In [28]:
print dp2+dp1

<xarray.DataArray ()>
array(-649529833.645227)
Coordinates:
    yt_ocean  float64 -57.23
    xt_ocean  float64 72.62
